# Python Requests Tag Workspace

Create, Write, Read tags in the Default or other workspace

## Imports

Import necessary python modules.

In [1]:
import os
import requests
import urllib3
from datetime import datetime
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

## Define Web Service Command Syntax for Requests

In [2]:
class WebServices(object):

    def __init__(self, host_url="", session=None):
        self.host_url = host_url
        self.session = session if session != None else requests.Session()

    def set_api_key(self, api_key):
        self.headers = {'x-ni-api-key':api_key}
        self.auth = ""
        
    def set_usr_pwd(self, username, password):
        self.headers = ""
        self.auth = (username, password)
    
    def post_route(self, url, post_json):
        print("POST " + self.host_url + url)
        response = self.session.post(self.host_url+url, json=post_json, verify=False, headers=self.headers, auth=self.auth)
        print(str(response))
        return response
    
    def get_route(self, url):
        print("GET " + self.host_url + url)
        response = self.session.get(self.host_url+url, json={}, verify=False, headers=self.headers, auth=self.auth)
        print(str(response))
        return response

    def put_route(self, url, put_json):
        print("PUT " + self.host_url + url)
        response = self.session.put(self.host_url+url, json=put_json, verify=False, headers=self.headers, auth=self.auth)
        print(str(response))
        return response

## Tag I/O Functions

In [3]:
def write_tag(webServices, tag_path, tag_properties, tag_value, tag_workspace):
    tag = {'collect_aggregates': False, 'keywords': None, 'path': tag_path, 'properties': tag_properties, 'type': 'STRING', 'workspace': tag_workspace}
    response = webServices.post_route("nitag/v2/tags", tag)
    if tag_workspace != None and tag_workspace != "":
        response = webServices.put_route("nitag/v2/tags/"+tag_workspace+"/"+tag_path+"/values/current", tag_value)
    else:
        response = webServices.put_route("nitag/v2/tags/"+tag_path+"/values/current", tag_value)

In [4]:
def read_tag(webServices, tag_path, tag_workspace):
    if tag_workspace != None and tag_workspace != "":
        response = webServices.get_route("nitag/v2/tags/"+tag_workspace+"/"+tag_path+"/values")
    else:
        response = webServices.get_route("nitag/v2/tags/"+tag_path+"/values")
    tag = response.json()
    return tag

In [5]:
def tag_io(webServices):
    now = datetime.today().isoformat() # "2019-09-05T18:45:08Z"
    tag_path = "NI_PXIe-8133_Embedded_Controller--MAC-00-80-2F-12-7F-31.Ownership.Status"
    tag_properties = {"IsConformed": "true", "Department": "OPS", "IsReleased": "true"}
    tag_value = {"value": {"type": "STRING", "value": "OPS"}, "timestamp": now}
    tag_workspace = None # workspace = Default
    #tag_workspace = "93f869bd-f4df-4fb6-84e3-0499cae06a7c" # workspace != Default
    write_tag(webServices, tag_path, tag_properties, tag_value, tag_workspace)
    tag = read_tag(webServices, tag_path, tag_workspace)
    print(str(tag))

## Connect to SystemLink, Run Tag I/O, Close Connection

In [6]:
# Connection Info
host_url = "https://localhost:443/"
host_url = os.getenv("SYSTEMLINK_HTTP_URI") # available only in JupyterHub on SystemLink server
api_key = os.getenv("SYSTEMLINK_API_KEY")   # available only in JupyterHub on SystemLink server

# Return WebServices object (set host_url)
webServices = WebServices(host_url)

# Define the connection credentials using an ApiKey or username/password
webServices.set_api_key(api_key)
#webServices.set_usr_pwd("admin", "mypassword")

# perform tag operations
tag_io(webServices)
print("done")

POST https://bradt2-lt/nitag/v2/tags
<Response [201]>
PUT https://bradt2-lt/nitag/v2/tags/NI_PXIe-8133_Embedded_Controller--MAC-00-80-2F-12-7F-31.Ownership.Status/values/current
<Response [202]>
GET https://bradt2-lt/nitag/v2/tags/NI_PXIe-8133_Embedded_Controller--MAC-00-80-2F-12-7F-31.Ownership.Status/values
<Response [200]>
{'current': {'timestamp': '2023-05-09T17:55:44.257948Z', 'value': {'type': 'STRING', 'value': 'OPS'}}, 'aggregates': None}
done
